# Импорт библиотек

import pickle
import random
import time
import os
import shutil
import copy

import numpy as np
import pandas as pd

from skimage import io
from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path
#import torchvision.transforms as transforms

from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from __future__ import print_function, division
from matplotlib import colors, pyplot as plt
%matplotlib inline
plt.ion()

import torch
import torch.nn as nn
import torchvision.models as models
import torchsummary
from torch.utils.data import datasets, DataLoader
import torchvision
from torchvision import datasets, transforms, models
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.autograd import Variable
import torch.onnx

In [51]:
import pandas as pd
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import numpy as np
import torchvision
import random
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import torch.onnx

In [2]:
DATA_MODES = ['train', 'val', 'test']
RESCALE_SIZE = 224
EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 0.001

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Модели

## Модель `create_nn_v2`

In [16]:
data_dir = './data/classed_nrj_train/'
test_dir = './data/test2/'

In [4]:
#Набор трансформаций для обучающей выборки
train_transforms = transforms.Compose([transforms.CenterCrop(224), 
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomVerticalFlip(p=0.5),
                                       transforms.RandomRotation(degrees=(45, 90)),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])
                                       ])

In [5]:
#Набор трансформаций для валидационной выборки
valid_transforms = transforms.Compose([transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           [0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225])
                                       ])

In [6]:
#Набор трансформаций для тестовой выборки
test_transforms = transforms.Compose([transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(
                                          [0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225])
                                      ])

In [7]:
#Объявляю функцию загрузчика с аргументами:
#Директория, Размер батча, Набор трансформаций для тренировочной выборки,
#Набор трансформаций для валидационной выборки, Размер валидационной выборки
def load_split_train_valid(datadir,
                           batch_size,
                           train_transforms,
                           valid_transforms,
                           valid_size): 
    
    #Загрузчик для тренировочной выборки
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    #Загрузчик для тренировочной выборки
    val_data = datasets.ImageFolder(datadir,
                    transform=valid_transforms)
    #Код для разделения на трейн и тест в указанном соотношении
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    train_idx, val_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    trainloader = DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size)
    valloader = DataLoader(val_data,
                   sampler=val_sampler, batch_size=batch_size)
    return trainloader, valloader

In [8]:
#Получаю тренировочный и валидационный генераторы
train_loader, val_loader = load_split_train_valid(datadir=data_dir,
                           batch_size=BATCH_SIZE,
                           train_transforms=train_transforms,
                           valid_transforms=valid_transforms,
                           valid_size = .2)
#Проверяю результаты работы генераторов
print(train_loader.dataset.classes)
print(val_loader.dataset.classes)
print(len(train_loader))
print(len(val_loader))

['1', '10', '20', '3', '30', '6']
['1', '10', '20', '3', '30', '6']
166
42


In [9]:
class ConvNet(nn.Module): 
        def __init__(self):
            super(ConvNet, self).__init__()
            self.conv1 = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.drop1 = nn.Dropout2d(p=0.25)
            self.conv2 = nn.Sequential(
                nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.drop2 = nn.Dropout2d(p=0.25)
            self.conv3 = nn.Sequential(
                nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.drop3 = nn.Dropout2d(p=0.25)
            self.conv4 = nn.Sequential(
                nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.fc1 = nn.Sequential(
                nn.Linear(18432, 2048),
                nn.BatchNorm1d(2048),
                nn.ReLU(),
            )
            self.fc2 = nn.Sequential(
                nn.Linear(2048, 1024),
                nn.BatchNorm1d(1024),
                nn.ReLU(),
            )
            self.out = nn.Linear(1024, 6)
            
            for m in self.modules():
                if isinstance(m, nn.Linear):
                    m.weight.data = nn.init.xavier_uniform_(m.weight.data,gain=nn.init.calculate_gain('relu'))
        
        def forward(self, x):
            x = self.drop1(self.conv1(x))
            x = self.drop2(self.conv2(x))
            x = self.drop3(self.conv3(x))
            x = self.conv4(x)
            x = x.view(x.size(0), -1)
            x = self.fc1(x)
            x = self.fc2(x)
            x = self.out(x)

            return F.log_softmax(x, dim=1)

In [10]:
net = ConvNet()
print(net)
net = net.cuda()

ConvNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop1): Dropout2d(p=0.25, inplace=False)
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop2): Dropout2d(p=0.25, inplace=False)
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop3): Dropout2d(p=0.25, inplace=False)
  (conv4): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Linear(in_features=18432, out_features=2048, bias=True)
    (1): BatchNorm1d(2048, 

In [11]:
#Задаю функцию потерь
criterion = nn.CrossEntropyLoss()
#Задаю оптимизатор
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)

total_step = len(train_loader)
for epoch in tqdm(range(EPOCHS)):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
    for i, (images, labels) in enumerate(train_loader):
    #Прямой запуск
        images=images.to(device)
        labels=labels.to(device)
            
        outputs = net(images)
        loss = criterion(outputs, labels)

        #Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        
        running_loss += loss.item() * images.size(0)
        running_corrects += (predicted == labels).sum().item()
        processed_data += images.size(0)


    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch + 1, EPOCHS, i + 1, total_step, running_loss / processed_data,
                  (running_corrects / processed_data) * 100))


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Epoch [1/10], Step [100/166], Loss: 2.0854, Accuracy: 22.14%


 10%|████████▎                                                                          | 1/10 [01:15<11:22, 75.82s/it]

Epoch [2/10], Step [100/166], Loss: 0.2361, Accuracy: 90.62%


 20%|████████████████▌                                                                  | 2/10 [02:29<09:56, 74.58s/it]

...

 80%|██████████████████████████████████████████████████████████████████▍                | 8/10 [09:47<02:26, 73.17s/it]

Epoch [9/10], Step [100/166], Loss: 0.0164, Accuracy: 99.42%


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [11:00<01:13, 73.21s/it]

Epoch [10/10], Step [100/166], Loss: 0.0311, Accuracy: 99.09%


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [12:14<00:00, 73.41s/it]


In [12]:
#Проверка на валидационной выборке
net.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in tqdm(val_loader):
        images=images.to(device)
        labels=labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Val Accuracy of the model on the test images: {} %'
              .format((correct / total) * 100))

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:11<00:00,  3.65it/s]

Test Accuracy of the model on the test images: 99.58584337349397 %


In [37]:
test_data = datasets.ImageFolder(test_dir,
                transform=test_transforms)

test_loader = DataLoader(test_data, shuffle=False, batch_size=BATCH_SIZE)
filenames = [line[0].split('\\')[1].split('.')[0] for line in test_data.imgs]
pred2class = [float(n) for n in train_loader.dataset.classes]
dict_pred = defaultdict(list)
dict_pred["id"] = filenames

In [61]:
%%time
net.eval()
with torch.no_grad():
    for i, (img, name) in enumerate(iter(test_loader)):
        outputs = net(img)
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.cpu()
        predicted = [pred2class[pred] for pred in predicted]
        dict_pred["classification_predictions"].extend(predicted)

Wall time: 4min 33s


In [64]:
data_frame = pd.DataFrame(dict_pred, columns=["id", "regression_predictions"])
data_frame.to_csv('./submissions/submission_tryCNN_reg.csv', index=False, header=True)